In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Import the data set

In [28]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data Cleaning

In [11]:
lab_data = pd.read_csv('../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data cleaning
erroneous = lab_data[lab_data['Thorax_length'] == '.']
erroneous

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
253,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,6,3,female,.,1.65,0.573,1.415,1.988,1.986,0.898,1.207,1.003,.


In [12]:
similar_rows = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Gogango_Creek')
    & (lab_data['Sex'] == 'female')
    & (lab_data['Thorax_length'] != ".")
]
similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])
similar_rows.head()

/var/folders/j_/9f1md2vd0_s6kpn12szf1t2c0000gn/T/ipykernel_67286/3776846164.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])


,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
162,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,1,female,1.166,1.910,0.578,1.675,2.254,2.252,0.982,1.381,1.123,1.932
164,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,2,female,1.174,1.829,0.622,1.549,2.170,2.170,0.997,1.360,1.117,1.848
166,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,3,female,1.199,1.864,0.585,1.659,2.244,2.242,1.009,1.376,1.142,1.870
167,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,1,female,1.186,1.872,0.628,1.571,2.199,2.199,1.002,1.356,1.143,1.854
169,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,2,female,1.176,1.872,0.630,1.579,2.207,2.207,0.966,1.345,1.103,1.876


In [13]:
similar_row_median = similar_rows['Thorax_length'].median()

lab_data['Thorax_length'] = lab_data.apply(
    lambda x: (x['Thorax_length'] if x['Thorax_length'] != "." else similar_row_median),
    axis=1
)
lab_data.head()

lab_data['wing_loading'] = lab_data.apply(
    lambda x: (x['wing_loading'] if x['wing_loading'] != "." else round((x['l3'] / x['Thorax_length']), 3)),
    axis=1
)

print(len(similar_rows))
print(similar_row_median)

lab_data.loc[253]

87
1.141


Species            D._aldrichi
Population       Gogango_Creek
Latitude                -23.77
Longitude               150.17
Year_start                1994
Year_end                  1994
Temperature                 25
Vial                         6
Replicate                    3
Sex                     female
Thorax_length            1.141
l2                        1.65
l3p                      0.573
l3d                      1.415
lpd                      1.988
l3                       1.986
w1                       0.898
w2                       1.207
w3                       1.003
wing_loading             1.741
Name: 253, dtype: object

# Model Building

In [29]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [30]:
lab_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1730 entries, 0 to 1730
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Species        1730 non-null   object 
 1   Population     1730 non-null   object 
 2   Latitude       1730 non-null   float64
 3   Longitude      1730 non-null   float64
 4   Year_start     1730 non-null   int64  
 5   Year_end       1730 non-null   int64  
 6   Temperature    1730 non-null   int64  
 7   Vial           1730 non-null   int64  
 8   Replicate      1730 non-null   int64  
 9   Sex            1730 non-null   object 
 10  Thorax_length  1730 non-null   float64
 11  l2             1730 non-null   float64
 12  l3p            1730 non-null   float64
 13  l3d            1730 non-null   float64
 14  lpd            1730 non-null   float64
 15  l3             1730 non-null   float64
 16  w1             1730 non-null   float64
 17  w2             1730 non-null   float64
 18  w3           

In [31]:
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
0,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,female,1.238,2.017,0.659,1.711,2.370,2.370,1.032,1.441,1.192,1.914
1,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,male,1.113,1.811,0.609,1.539,2.148,2.146,0.938,1.299,1.066,1.928
2,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,female,1.215,1.985,0.648,1.671,2.319,2.319,0.991,1.396,1.142,1.908
3,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,male,1.123,1.713,0.596,1.495,2.091,2.088,0.958,1.286,1.062,1.860
4,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,2,1,female,1.218,1.938,0.641,1.658,2.298,2.298,1.010,1.418,1.148,1.886


In [32]:
#TODO: Currently using decision tree so no need to normalise, but must do for other methods

In [33]:
lab_data = shuffle(lab_data)
lab_data['class'] = lab_data.apply(
    lambda x: f"{x['Population']}, {x['Sex']}",
    axis=1
)
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class
1726,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,30,10,1,male,...,1.568,0.528,1.309,1.837,1.837,0.783,1.107,0.920,1.778,"Wahruna, male"
368,D._aldrichi,Grandchester,-27.68,152.45,1994,1994,20,6,1,male,...,1.829,0.587,1.560,2.146,2.145,0.950,1.309,1.110,1.935,"Grandchester, male"
14,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,4,1,male,...,1.845,0.632,1.539,2.170,2.168,0.944,1.318,1.093,1.855,"Binjour, male"
88,D._aldrichi,Binjour,-25.52,151.45,1994,1994,25,8,1,female,...,1.685,0.561,1.508,2.068,2.066,0.934,1.241,1.055,1.790,"Binjour, female"
700,D._aldrichi,Wahruna,-25.20,151.17,1994,1994,20,6,1,female,...,1.902,0.677,1.605,2.282,2.282,1.047,1.382,1.196,1.936,"Wahruna, female"


In [34]:

train, test = train_test_split(lab_data, test_size=0.3)

In [35]:
classifier = DecisionTreeClassifier(criterion='entropy', max_depth=5)
#TODO: Add Thorax Length and wing_loading to this training
classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [36]:
train['class_predictions'] = classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [37]:
train.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class,class_predictions
1365,D._buzzatii,Grandchester,-27.68,152.45,1994,1994,30,9,3,female,...,0.567,1.456,2.021,2.019,0.883,1.214,1.013,1.753,"Grandchester, female","Oxford_Downs, female"
1624,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,25,2,3,male,...,0.573,1.458,2.031,2.030,0.880,1.249,0.991,1.798,"Wahruna, male","Oxford_Downs, male"
467,D._aldrichi,Grandchester,-27.68,152.45,1994,1994,30,3,3,female,...,0.570,1.319,1.889,1.887,0.855,1.162,0.959,1.663,"Grandchester, female","Grandchester, female"
1096,D._buzzatii,Gogango_Creek,-23.77,150.17,1994,1994,25,3,3,male,...,0.548,1.428,1.975,1.974,0.851,1.201,0.962,1.800,"Gogango_Creek, male","Oxford_Downs, male"
810,D._aldrichi,Wahruna,-25.20,151.17,1994,1994,30,5,2,female,...,0.563,1.367,1.930,1.929,0.845,1.168,0.958,1.747,"Wahruna, female","Wahruna, female"


In [38]:
#Correct predictions
train.apply(
    lambda x: 1 if x['class_predictions'] == x['class'] else 0,
    axis=1
).sum()

349

In [39]:
#Incorrect predictions
train.apply(
    lambda x: 1 if x['class_predictions'] != x['class'] else 0,
    axis=1
).sum()

862

In [40]:
# Retry with a deeper depth

In [41]:
deep_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=10)

deep_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [42]:
train['deep_class_predictions'] = deep_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [43]:
#Correct deep predictions
train.apply(
    lambda x: 1 if x['deep_class_predictions'] == x['class'] else 0,
    axis=1
).sum()

828

In [44]:
min_leaf_classifier = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

min_leaf_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

In [45]:
train['min_leaf_predictions'] = min_leaf_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [46]:
#Correct min_leaf predictions
train.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

779

In [47]:
fully_fit_classifier = DecisionTreeClassifier(criterion='entropy')

fully_fit_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy')

In [48]:
train['fully_fit_predictions'] = fully_fit_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [49]:
#Correct fully_fit predictions
train.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

1211

In [50]:
# Checking the min leaf and fully fit classifiers against the test set

In [51]:
#Correst min_leaf predictions on the test set

In [52]:
test['min_leaf_predictions'] = min_leaf_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['fully_fit_predictions'] = fully_fit_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])


correct_min_leaf_classification_test = test.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Min leaf correct predictions test set:" + str(correct_min_leaf_classification_test))

correct_fully_fit_classification_test = test.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Fully fit predictions test set:" + str(correct_fully_fit_classification_test))


Min leaf correct predictions test set:106
Fully fit predictions test set:100


In [53]:
print(f"accuracy: {correct_min_leaf_classification_test / len(test)}")

accuracy: 0.20423892100192678
